In [31]:
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import XMLOutputParser, JsonOutputParser, PydanticOutputParser

load_dotenv()

True

In [57]:
model = ChatGroq(model="gemma2-9b-it")

In [ ]:
prompt_chat = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        ("user", "{input}")
    ]
)

In [ ]:
chain = prompt_chat | model
response = chain.invoke({"input": "What is the capital of France?"})
response

AIMessage(content='The capital of France is **Paris**.  🗼🇫🇷 \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 22, 'total_tokens': 38, 'completion_time': 0.029090909, 'prompt_time': 0.002139225, 'queue_time': 0.015195203999999999, 'total_time': 0.031230134}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--b19f77ea-c821-4d36-8f55-5f98242fedcb-0', usage_metadata={'input_tokens': 22, 'output_tokens': 16, 'total_tokens': 38})

In [25]:
json_parser = JsonOutputParser()

prompt = PromptTemplate(
    template="What is the capital of {input}? {format_instructions}",
    partial_variables={
        "format_instructions": json_parser.get_format_instructions(),
    },
    input_variables=["input"],
)

chain = prompt | model | json_parser
response = chain.invoke({"input": "What is the capital of France?"})
response

{'capital': 'Paris'}

In [30]:
from pydantic import BaseModel
class MyOutputModel(BaseModel):
    country: str
    capital: str
    latitude: float
    longitude: float
    timezone: str

pydantic_parser = PydanticOutputParser(pydantic_object=MyOutputModel)

prompt = PromptTemplate(
    template="You are an helpful AI assisntant. Answer the user question: {input}? return {format_instructions}",
    partial_variables={
        "format_instructions": pydantic_parser.get_format_instructions(),
    },
    input_variables=["input"],
)

chain = prompt | model | pydantic_parser
response = chain.invoke({"input": "What is the capital of France?"})
response

MyOutputModel(country='France', capital='Paris', latitude=48.8566, longitude=2.3522, timezone='Europe/Paris')

In [46]:
xml_parser = XMLOutputParser(tags=["movies", "actor", "film", "name", "genre"])

prompt = PromptTemplate(
    template="You are an helpful AI assisntant. Answer the user question: {input}? return {format_instructions}",
    partial_variables={
        "format_instructions": xml_parser.get_format_instructions(),
    },
    input_variables=["input"],
)

chain = prompt | model | xml_parser
response = chain.invoke({"input": "tell me some movies with Tom Hanks"})
response

{'movies': [{'actor': 'Tom Hanks'},
  {'film': [{'name': 'Forrest Gump'}, {'genre': 'Drama, Romance'}]},
  {'film': [{'name': 'Saving Private Ryan'}, {'genre': 'War, Drama'}]},
  {'film': [{'name': 'Toy Story'}, {'genre': 'Animation, Adventure, Comedy'}]},
  {'film': [{'name': 'Cast Away'}, {'genre': 'Adventure, Drama'}]}]}

In [110]:
from pydantic import Field, BaseModel
from typing import Optional

class Product(BaseModel):
    name: str = Field(description="The name of the product")
    tentative_price: Optional[float] = Field(default=None, description="The price of the product in USD")
    description: str = Field(description="A brief description of the product")


In [111]:
from langchain_core.prompts import  ChatPromptTemplate, PromptTemplate, SystemMessagePromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.messages import HumanMessage

product_parser = PydanticOutputParser(pydantic_object=Product)

_prompt_template = PromptTemplate(
    template="You are a helpful eCommerce Product assistant. Answer the user question: {input}. Return the product information in the format: {format_instructions}",
    input_variables=["input"],
    partial_variables={"format_instructions": product_parser.get_format_instructions()}
)

In [112]:
prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate(prompt=_prompt_template),
        HumanMessage(content="{input}"),
    ]
)

chain = prompt | model | product_parser

response = chain.invoke({"input": "Tell me about the latest iPhone."})
response

Product(name='iPhone 15 Pro Max', tentative_price=1099.0, description='The latest iPhone, featuring an advanced camera system, a stunning display, and powerful performance.')